# Análisis Automatizado Avanzado de Calidad de Vinos Blancos

**Proyecto de Minería de Datos - Análisis Exhaustivo**

In [ ]:
#!/usr/bin/env python3
"""
ANÁLISIS AUTOMATIZADO AVANZADO DE CALIDAD DE VINOS BLANCOS
Proyecto de Minería de Datos

Este script realiza:
- Análisis exploratorio completo
- Tests estadísticos
- Machine Learning
- Clustering y PCA
- Generación de insights con Claude API
"""

## 1. IMPORTAR LIBRERÍAS

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from scipy.stats import shapiro, normaltest, f_oneway, kruskal

# Machine Learning
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import (classification_report, confusion_matrix, accuracy_score, 
                             precision_score, recall_score, f1_score,
                             mean_squared_error, mean_absolute_error, r2_score,
                             silhouette_score)
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans

import warnings
import os
from dotenv import load_dotenv

# Configuración
warnings.filterwarnings('ignore')
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette('husl')
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', '{:.4f}'.format)

load_dotenv()
RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)

# Crear directorios
os.makedirs('outputs', exist_ok=True)
os.makedirs('reports', exist_ok=True)

print("="*70)
print("ANÁLISIS AUTOMATIZADO DE CALIDAD DE VINOS BLANCOS")
print("="*70)

ANÁLISIS AUTOMATIZADO DE CALIDAD DE VINOS BLANCOS


## 2. CARGAR Y VALIDAR DATOS

In [3]:
print("\n[1/16] Cargando datos...")
df = pd.read_csv('data/winequality-white.csv')
df = df.dropna(axis=1, how='all')
df.columns = df.columns.str.strip()

info_dataset = {
    'Nombre': 'Wine Quality - White Wine',
    'Registros': df.shape[0],
    'Columnas': df.shape[1],
    'Duplicados': df.duplicated().sum()
}

columnas_numericas = df.select_dtypes(include=[np.number]).columns.tolist()
columnas_continuas = [col for col in columnas_numericas if col != 'quality']

print(f"   Dataset: {info_dataset['Registros']:,} registros x {info_dataset['Columnas']} columnas")


[1/16] Cargando datos...
   Dataset: 4,898 registros x 12 columnas


## 3. ANÁLISIS DE VALORES FALTANTES

In [16]:
print("[2/16] Analizando valores faltantes...")
valores_faltantes = df.isnull().sum()
porcentaje_faltantes = (df.isnull().sum() / len(df)) * 100

# Heatmap de valores faltantes
fig, ax = plt.subplots(figsize=(14, 6))
sns.heatmap(df.isnull(), cbar=True, yticklabels=False, cmap='viridis', ax=ax)
ax.set_title('Mapa de Valores Faltantes', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.savefig('outputs/01_heatmap_valores_faltantes_v2.png', dpi=150, bbox_inches='tight')
plt.close()

[2/16] Analizando valores faltantes...


## 4. ESTADÍSTICAS DESCRIPTIVAS

In [6]:
print("[3/16] Calculando estadísticas descriptivas...")
estadisticas = df.describe().T
estadisticas['mediana'] = df.median()
estadisticas['moda'] = df.mode().iloc[0]
estadisticas['varianza'] = df.var()
estadisticas['rango'] = estadisticas['max'] - estadisticas['min']
estadisticas['IQR'] = df.quantile(0.75) - df.quantile(0.25)
estadisticas['coef_var (%)'] = (estadisticas['std'] / estadisticas['mean']) * 100
estadisticas['asimetria'] = df.skew()
estadisticas['curtosis'] = df.kurtosis()

# Distribución de calidad
frecuencias = df['quality'].value_counts().sort_index()
porcentajes = (frecuencias / len(df) * 100).round(2)
acumulado = porcentajes.cumsum()

df_quality = pd.DataFrame({
    'Calidad': frecuencias.index,
    'Frecuencia': frecuencias.values,
    'Porcentaje (%)': porcentajes.values,
    'Acumulado (%)': acumulado.values
})

# Crear categorías de calidad
def categorizar_calidad(q):
    if q <= 4: return 'Baja'
    elif q <= 6: return 'Media'
    else: return 'Alta'

df['quality_category'] = df['quality'].apply(categorizar_calidad)

[3/16] Calculando estadísticas descriptivas...


## 5. TESTS DE NORMALIDAD

In [17]:
print("[4/16] Ejecutando tests de normalidad...")
resultados_normalidad = []

for col in columnas_continuas:
    muestra = df[col].dropna().sample(min(5000, len(df)), random_state=RANDOM_STATE)
    stat_shapiro, p_shapiro = shapiro(muestra)
    stat_dagostino, p_dagostino = normaltest(df[col].dropna())
    es_normal = "Sí" if (p_shapiro > 0.05 and p_dagostino > 0.05) else "No"
    
    resultados_normalidad.append({
        'Variable': col,
        'Shapiro-Wilk (p)': p_shapiro,
        "D'Agostino (p)": p_dagostino,
        'Normal': es_normal
    })

df_normalidad = pd.DataFrame(resultados_normalidad)

# Q-Q plots
n_cols = 3
n_rows = (len(columnas_continuas) + n_cols - 1) // n_cols
fig, axes = plt.subplots(n_rows, n_cols, figsize=(15, 4*n_rows))
axes = axes.flatten()

for i, col in enumerate(columnas_continuas):
    stats.probplot(df[col].dropna(), dist="norm", plot=axes[i])
    axes[i].set_title(f'Q-Q Plot: {col}', fontsize=10, fontweight='bold')
for j in range(i+1, len(axes)):
    axes[j].set_visible(False)

plt.suptitle('Q-Q Plots para Verificación de Normalidad', fontsize=14, fontweight='bold', y=1.02)
plt.tight_layout()
plt.savefig('outputs/02_qq_plots_v2.png', dpi=150, bbox_inches='tight')
plt.close()

[4/16] Ejecutando tests de normalidad...


## 6. VISUALIZACIONES

In [18]:
print("[5/16] Generando visualizaciones...")

# Histogramas con KDE
fig, axes = plt.subplots(n_rows, n_cols, figsize=(16, 4*n_rows))
axes = axes.flatten()
for i, col in enumerate(columnas_continuas):
    sns.histplot(df[col].dropna(), bins=40, kde=True, ax=axes[i], color='steelblue', alpha=0.7)
    axes[i].axvline(df[col].mean(), color='red', linestyle='--', linewidth=2, label=f'Media: {df[col].mean():.2f}')
    axes[i].axvline(df[col].median(), color='green', linestyle=':', linewidth=2, label=f'Mediana: {df[col].median():.2f}')
    axes[i].set_title(f'{col}', fontsize=11, fontweight='bold')
    axes[i].legend(fontsize=8)
for j in range(i+1, len(axes)):
    axes[j].set_visible(False)
plt.suptitle('Distribución de Variables (Histogramas + KDE)', fontsize=14, fontweight='bold', y=1.02)
plt.tight_layout()
plt.savefig('outputs/03_histogramas_kde_v2.png', dpi=150, bbox_inches='tight')
plt.close()

# Boxplots
fig, axes = plt.subplots(n_rows, n_cols, figsize=(16, 4*n_rows))
axes = axes.flatten()
for i, col in enumerate(columnas_continuas):
    bp = axes[i].boxplot(df[col].dropna(), patch_artist=True)
    bp['boxes'][0].set_facecolor('lightblue')
    Q1, Q3 = df[col].quantile(0.25), df[col].quantile(0.75)
    IQR = Q3 - Q1
    outliers = ((df[col] < Q1 - 1.5*IQR) | (df[col] > Q3 + 1.5*IQR)).sum()
    axes[i].set_title(f'{col}\n({outliers} outliers)', fontsize=10, fontweight='bold')
for j in range(i+1, len(axes)):
    axes[j].set_visible(False)
plt.suptitle('Boxplots - Detección de Outliers (Método IQR)', fontsize=14, fontweight='bold', y=1.02)
plt.tight_layout()
plt.savefig('outputs/04_boxplots_v2.png', dpi=150, bbox_inches='tight')
plt.close()

# Violin plots por categoría
fig, axes = plt.subplots(n_rows, n_cols, figsize=(16, 4*n_rows))
axes = axes.flatten()
orden_categorias = ['Baja', 'Media', 'Alta']
for i, col in enumerate(columnas_continuas):
    sns.violinplot(data=df, x='quality_category', y=col, order=orden_categorias, palette='RdYlGn', ax=axes[i])
    axes[i].set_title(f'{col}', fontsize=10, fontweight='bold')
for j in range(i+1, len(axes)):
    axes[j].set_visible(False)
plt.suptitle('Distribución por Categoría de Calidad', fontsize=14, fontweight='bold', y=1.02)
plt.tight_layout()
plt.savefig('outputs/05_violin_plots_v2.png', dpi=150, bbox_inches='tight')
plt.close()

# Barras de quality
fig, axes = plt.subplots(1, 2, figsize=(14, 5))
colores = sns.color_palette('RdYlGn', n_colors=len(frecuencias))
bars = axes[0].bar(frecuencias.index.astype(str), frecuencias.values, color=colores, edgecolor='black')
for bar, freq, pct in zip(bars, frecuencias.values, porcentajes.values):
    axes[0].text(bar.get_x() + bar.get_width()/2., bar.get_height(), f'{freq}\n({pct}%)', ha='center', va='bottom', fontsize=9)
axes[0].set_title('Distribución por Nivel de Calidad', fontsize=12, fontweight='bold')

cat_counts = df['quality_category'].value_counts()[orden_categorias]
colores_cat = ['#d73027', '#fee08b', '#1a9850']
bars2 = axes[1].bar(cat_counts.index, cat_counts.values, color=colores_cat, edgecolor='black')
for bar, freq in zip(bars2, cat_counts.values):
    pct = freq / len(df) * 100
    axes[1].text(bar.get_x() + bar.get_width()/2., bar.get_height(), f'{freq}\n({pct:.1f}%)', ha='center', va='bottom')
axes[1].set_title('Distribución por Categoría', fontsize=12, fontweight='bold')
plt.tight_layout()
plt.savefig('outputs/06_barras_quality_v2.png', dpi=150, bbox_inches='tight')
plt.close()

[5/16] Generando visualizaciones...


## 7. ANÁLISIS DE OUTLIERS

In [10]:
print("[6/16] Analizando outliers...")
resultados_outliers = []
for col in columnas_continuas:
    Q1, Q3 = df[col].quantile(0.25), df[col].quantile(0.75)
    IQR = Q3 - Q1
    outliers_iqr = ((df[col] < Q1 - 1.5*IQR) | (df[col] > Q3 + 1.5*IQR)).sum()
    z_scores = np.abs(stats.zscore(df[col].dropna()))
    outliers_zscore = (z_scores > 3).sum()
    
    resultados_outliers.append({
        'Variable': col,
        'Q1': Q1, 'Q3': Q3, 'IQR': IQR,
        'Límite Inferior': Q1 - 1.5*IQR,
        'Límite Superior': Q3 + 1.5*IQR,
        'Outliers (IQR)': outliers_iqr,
        '% Outliers': (outliers_iqr / len(df)) * 100
    })

df_outliers = pd.DataFrame(resultados_outliers)
total_outliers_iqr = df_outliers['Outliers (IQR)'].sum()

[6/16] Analizando outliers...


## 8. ANÁLISIS DE CORRELACIONES

In [19]:
print("[7/16] Calculando correlaciones...")
corr_pearson = df[columnas_numericas].corr(method='pearson')
corr_spearman = df[columnas_numericas].corr(method='spearman')

# Heatmaps
fig, axes = plt.subplots(1, 2, figsize=(18, 7))
mask = np.triu(np.ones_like(corr_pearson, dtype=bool))
sns.heatmap(corr_pearson, mask=mask, annot=True, fmt='.2f', cmap='RdBu_r', center=0, square=True, ax=axes[0])
axes[0].set_title('Correlación de Pearson', fontsize=12, fontweight='bold')
sns.heatmap(corr_spearman, mask=mask, annot=True, fmt='.2f', cmap='RdBu_r', center=0, square=True, ax=axes[1])
axes[1].set_title('Correlación de Spearman', fontsize=12, fontweight='bold')
plt.tight_layout()
plt.savefig('outputs/07_heatmaps_correlacion_v2.png', dpi=150, bbox_inches='tight')
plt.close()

# Top correlaciones
def obtener_top_correlaciones(matriz_corr, n=10):
    pares = []
    for i in range(len(matriz_corr.columns)):
        for j in range(i+1, len(matriz_corr.columns)):
            col1, col2 = matriz_corr.columns[i], matriz_corr.columns[j]
            corr = matriz_corr.iloc[i, j]
            pares.append((col1, col2, corr, abs(corr)))
    pares.sort(key=lambda x: x[3], reverse=True)
    return pares[:n]

top_corr = obtener_top_correlaciones(corr_pearson, 10)
df_top_corr = pd.DataFrame(top_corr, columns=['Variable 1', 'Variable 2', 'Correlación', '|Correlación|'])
df_top_corr.index = range(1, len(df_top_corr) + 1)

# Correlaciones con quality
corr_con_quality = corr_pearson['quality'].drop('quality').sort_values(key=abs, ascending=False)
df_corr_quality = pd.DataFrame({
    'Variable': corr_con_quality.index,
    'Pearson': corr_con_quality.values,
    'Spearman': corr_spearman['quality'].drop('quality')[corr_con_quality.index].values
})
df_corr_quality.index = range(1, len(df_corr_quality) + 1)

# Gráfico correlaciones con quality
fig, ax = plt.subplots(figsize=(12, 6))
colors = ['green' if c > 0 else 'red' for c in corr_con_quality.values]
bars = ax.barh(corr_con_quality.index, corr_con_quality.values, color=colors, edgecolor='black', alpha=0.7)
ax.axvline(x=0, color='black', linewidth=0.5)
ax.set_title('Correlación de Variables con Calidad del Vino', fontsize=14, fontweight='bold')
for bar, val in zip(bars, corr_con_quality.values):
    ax.text(val + 0.02 if val > 0 else val - 0.02, bar.get_y() + bar.get_height()/2, f'{val:.3f}', va='center', fontsize=9)
plt.tight_layout()
plt.savefig('outputs/08_correlacion_con_quality_v2.png', dpi=150, bbox_inches='tight')
plt.close()

[7/16] Calculando correlaciones...


## 9. TESTS ESTADÍSTICOS

In [13]:
print("[8/16] Ejecutando tests estadísticos...")
resultados_tests = []
for col in columnas_continuas:
    grupos = [df[df['quality'] == q][col].dropna() for q in sorted(df['quality'].unique())]
    stat_anova, p_anova = f_oneway(*grupos)
    stat_kruskal, p_kruskal = kruskal(*grupos)
    
    resultados_tests.append({
        'Variable': col,
        'ANOVA (F)': stat_anova,
        'ANOVA (p)': p_anova,
        'Kruskal (H)': stat_kruskal,
        'Kruskal (p)': p_kruskal,
        'Significativo': "Sí" if p_kruskal < 0.05 else "No"
    })

df_tests = pd.DataFrame(resultados_tests)
vars_significativas = df_tests[df_tests['Significativo'] == 'Sí']['Variable'].tolist()

[8/16] Ejecutando tests estadísticos...


## 10. ESTADÍSTICAS POR GRUPO

In [14]:
print("[9/16] Calculando estadísticas por grupo...")
stats_por_calidad = df.groupby('quality')[columnas_continuas].agg(['mean', 'std', 'median'])

df_baja = df[df['quality'] <= 4]
df_alta = df[df['quality'] >= 7]
comparacion = pd.DataFrame({
    'Variable': columnas_continuas,
    'Media (Baja)': df_baja[columnas_continuas].mean().values,
    'Media (Alta)': df_alta[columnas_continuas].mean().values,
    'Diferencia': (df_alta[columnas_continuas].mean() - df_baja[columnas_continuas].mean()).values,
    'Diferencia (%)': ((df_alta[columnas_continuas].mean() - df_baja[columnas_continuas].mean()) / df_baja[columnas_continuas].mean() * 100).values
})

[9/16] Calculando estadísticas por grupo...


## 11. PCA

In [20]:
print("[10/16] Ejecutando PCA...")
scaler = StandardScaler()
X_scaled = scaler.fit_transform(df[columnas_continuas])

pca = PCA()
X_pca = pca.fit_transform(X_scaled)
varianza_explicada = pca.explained_variance_ratio_
varianza_acumulada = np.cumsum(varianza_explicada)

n_comp_80 = np.argmax(varianza_acumulada >= 0.80) + 1
n_comp_95 = np.argmax(varianza_acumulada >= 0.95) + 1

# Gráficos PCA
fig, axes = plt.subplots(1, 2, figsize=(14, 5))
axes[0].bar(range(1, len(varianza_explicada)+1), varianza_explicada * 100, alpha=0.7, color='steelblue')
axes[0].plot(range(1, len(varianza_explicada)+1), varianza_explicada * 100, 'ro-')
axes[0].set_title('Scree Plot', fontsize=12, fontweight='bold')
axes[1].plot(range(1, len(varianza_acumulada)+1), varianza_acumulada * 100, 'bo-')
axes[1].axhline(y=80, color='red', linestyle='--', label='80%')
axes[1].axhline(y=95, color='green', linestyle='--', label='95%')
axes[1].set_title('Varianza Acumulada', fontsize=12, fontweight='bold')
axes[1].legend()
plt.tight_layout()
plt.savefig('outputs/09_pca_varianza_v2.png', dpi=150, bbox_inches='tight')
plt.close()

# PCA scatter
fig, ax = plt.subplots(figsize=(10, 8))
scatter = ax.scatter(X_pca[:, 0], X_pca[:, 1], c=df['quality'], cmap='RdYlGn', alpha=0.6, s=30)
ax.set_xlabel(f'PC1 ({varianza_explicada[0]*100:.1f}%)')
ax.set_ylabel(f'PC2 ({varianza_explicada[1]*100:.1f}%)')
ax.set_title('Proyección PCA', fontsize=14, fontweight='bold')
plt.colorbar(scatter, label='Calidad')
plt.tight_layout()
plt.savefig('outputs/10_pca_scatter_v2.png', dpi=150, bbox_inches='tight')
plt.close()

# Loadings
loadings = pd.DataFrame(pca.components_.T, columns=[f'PC{i+1}' for i in range(len(columnas_continuas))], index=columnas_continuas)

[10/16] Ejecutando PCA...


## 12. CLUSTERING

In [21]:
print("[11/16] Ejecutando clustering...")
inertias, silhouettes = [], []
K_range = range(2, 11)
for k in K_range:
    kmeans = KMeans(n_clusters=k, random_state=RANDOM_STATE, n_init=10)
    kmeans.fit(X_scaled)
    inertias.append(kmeans.inertia_)
    silhouettes.append(silhouette_score(X_scaled, kmeans.labels_))

mejor_k = K_range[np.argmax(silhouettes)]

fig, axes = plt.subplots(1, 2, figsize=(14, 5))
axes[0].plot(K_range, inertias, 'bo-')
axes[0].set_title('Método del Codo', fontweight='bold')
axes[1].plot(K_range, silhouettes, 'go-')
axes[1].set_title('Silhouette Score', fontweight='bold')
plt.tight_layout()
plt.savefig('outputs/11_clustering_elbow_v2.png', dpi=150, bbox_inches='tight')
plt.close()

n_clusters = 4
kmeans_final = KMeans(n_clusters=n_clusters, random_state=RANDOM_STATE, n_init=10)
df['cluster'] = kmeans_final.fit_predict(X_scaled)

fig, ax = plt.subplots(figsize=(10, 8))
scatter = ax.scatter(X_pca[:, 0], X_pca[:, 1], c=df['cluster'], cmap='viridis', alpha=0.6, s=30)
centroides_pca = pca.transform(kmeans_final.cluster_centers_)
ax.scatter(centroides_pca[:, 0], centroides_pca[:, 1], c='red', marker='X', s=200, edgecolors='black')
ax.set_title(f'Clusters K-Means (K={n_clusters})', fontsize=14, fontweight='bold')
plt.colorbar(scatter, label='Cluster')
plt.tight_layout()
plt.savefig('outputs/12_clustering_pca_v2.png', dpi=150, bbox_inches='tight')
plt.close()

perfil_clusters = df.groupby('cluster')[columnas_continuas + ['quality']].mean()

[11/16] Ejecutando clustering...


## 13. MACHINE LEARNING - CLASIFICACIÓN

In [22]:
print("[12/16] Entrenando modelos de clasificación...")
X = df[columnas_continuas]
y = df['quality']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=RANDOM_STATE, stratify=y)

scaler_ml = StandardScaler()
X_train_scaled = scaler_ml.fit_transform(X_train)
X_test_scaled = scaler_ml.transform(X_test)

modelos = {
    'Logistic Regression': LogisticRegression(max_iter=1000, random_state=RANDOM_STATE),
    'Random Forest': RandomForestClassifier(n_estimators=100, random_state=RANDOM_STATE),
    'Gradient Boosting': GradientBoostingClassifier(n_estimators=100, random_state=RANDOM_STATE),
    'KNN': KNeighborsClassifier(n_neighbors=5),
    'Decision Tree': DecisionTreeClassifier(random_state=RANDOM_STATE)
}

resultados_modelos = []
for nombre, modelo in modelos.items():
    modelo.fit(X_train_scaled, y_train)
    y_pred = modelo.predict(X_test_scaled)
    cv_scores = cross_val_score(modelo, X_train_scaled, y_train, cv=5)
    
    resultados_modelos.append({
        'Modelo': nombre,
        'Accuracy': accuracy_score(y_test, y_pred),
        'Precision': precision_score(y_test, y_pred, average='weighted', zero_division=0),
        'Recall': recall_score(y_test, y_pred, average='weighted', zero_division=0),
        'F1-Score': f1_score(y_test, y_pred, average='weighted', zero_division=0),
        'CV Mean': cv_scores.mean(),
        'CV Std': cv_scores.std()
    })

df_resultados_ml = pd.DataFrame(resultados_modelos).sort_values('Accuracy', ascending=False)

# Matriz de confusión (mejor modelo)
mejor_modelo = modelos['Random Forest']
y_pred_best = mejor_modelo.predict(X_test_scaled)

fig, ax = plt.subplots(figsize=(10, 8))
cm = confusion_matrix(y_test, y_pred_best)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', ax=ax, xticklabels=sorted(y.unique()), yticklabels=sorted(y.unique()))
ax.set_title('Matriz de Confusión - Random Forest', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.savefig('outputs/13_confusion_matrix_v2.png', dpi=150, bbox_inches='tight')
plt.close()

# Comparación de modelos
fig, ax = plt.subplots(figsize=(12, 6))
x_pos = np.arange(len(df_resultados_ml))
width = 0.2
ax.bar(x_pos - width*1.5, df_resultados_ml['Accuracy'], width, label='Accuracy', color='steelblue')
ax.bar(x_pos - width/2, df_resultados_ml['Precision'], width, label='Precision', color='green')
ax.bar(x_pos + width/2, df_resultados_ml['Recall'], width, label='Recall', color='orange')
ax.bar(x_pos + width*1.5, df_resultados_ml['F1-Score'], width, label='F1-Score', color='red')
ax.set_xticks(x_pos)
ax.set_xticklabels(df_resultados_ml['Modelo'], rotation=45, ha='right')
ax.legend()
ax.set_ylim(0, 1)
ax.set_title('Comparación de Modelos', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.savefig('outputs/14_comparacion_modelos_v2.png', dpi=150, bbox_inches='tight')
plt.close()

[12/16] Entrenando modelos de clasificación...


## 14. FEATURE IMPORTANCE

In [23]:
print("[13/16] Calculando feature importance...")
importancias = mejor_modelo.feature_importances_
indices = np.argsort(importancias)[::-1]
df_importancia = pd.DataFrame({
    'Variable': [columnas_continuas[i] for i in indices],
    'Importancia': [importancias[i] for i in indices],
    'Importancia (%)': [importancias[i] * 100 for i in indices]
})
df_importancia.index = range(1, len(df_importancia) + 1)

fig, ax = plt.subplots(figsize=(10, 8))
colors = plt.cm.RdYlGn(np.linspace(0.2, 0.8, len(df_importancia)))
bars = ax.barh(df_importancia['Variable'], df_importancia['Importancia'], color=colors, edgecolor='black')
ax.set_title('Feature Importance - Random Forest', fontsize=14, fontweight='bold')
for bar, val in zip(bars, df_importancia['Importancia']):
    ax.text(val + 0.005, bar.get_y() + bar.get_height()/2, f'{val:.3f}', va='center', fontsize=9)
plt.gca().invert_yaxis()
plt.tight_layout()
plt.savefig('outputs/15_feature_importance_v2.png', dpi=150, bbox_inches='tight')
plt.close()

[13/16] Calculando feature importance...


## 15. REGRESIÓN

In [24]:
print("[14/16] Entrenando modelos de regresión...")
reg_lr = LinearRegression()
reg_rf = RandomForestRegressor(n_estimators=100, random_state=RANDOM_STATE)

reg_lr.fit(X_train_scaled, y_train)
reg_rf.fit(X_train_scaled, y_train)

y_pred_lr = reg_lr.predict(X_test_scaled)
y_pred_rf = reg_rf.predict(X_test_scaled)

resultados_reg = []
for nombre, y_pred_reg in [('Linear Regression', y_pred_lr), ('Random Forest Regressor', y_pred_rf)]:
    resultados_reg.append({
        'Modelo': nombre,
        'MSE': mean_squared_error(y_test, y_pred_reg),
        'RMSE': np.sqrt(mean_squared_error(y_test, y_pred_reg)),
        'MAE': mean_absolute_error(y_test, y_pred_reg),
        'R²': r2_score(y_test, y_pred_reg)
    })
df_resultados_reg = pd.DataFrame(resultados_reg)

fig, axes = plt.subplots(1, 2, figsize=(14, 5))
axes[0].scatter(y_test, y_pred_lr, alpha=0.5, color='blue')
axes[0].plot([y.min(), y.max()], [y.min(), y.max()], 'r--')
axes[0].set_title(f'Linear Regression (R² = {r2_score(y_test, y_pred_lr):.3f})', fontweight='bold')
axes[1].scatter(y_test, y_pred_rf, alpha=0.5, color='green')
axes[1].plot([y.min(), y.max()], [y.min(), y.max()], 'r--')
axes[1].set_title(f'Random Forest (R² = {r2_score(y_test, y_pred_rf):.3f})', fontweight='bold')
plt.tight_layout()
plt.savefig('outputs/16_regression_predictions_v2.png', dpi=150, bbox_inches='tight')
plt.close()

coef_lr = pd.DataFrame({
    'Variable': columnas_continuas,
    'Coeficiente': reg_lr.coef_
}).sort_values('Coeficiente', key=abs, ascending=False)
coef_lr.index = range(1, len(coef_lr) + 1)

[14/16] Entrenando modelos de regresión...


## 16. RESUMEN PARA CLAUDE

In [25]:
print("[15/16] Generando resumen...")

resumen_completo = f"""
================================================================================
RESUMEN COMPLETO DEL ANÁLISIS DE DATOS - WINE QUALITY (WHITE WINE)
================================================================================

1. INFORMACIÓN DEL DATASET
--------------------------------------------------------------------------------
   - Nombre: Wine Quality - White Wine
   - Fuente: UCI Machine Learning Repository
   - Dimensiones: {df.shape[0]:,} registros x {df.shape[1]} columnas
   - Variables continuas: {len(columnas_continuas)}
   - Variable objetivo: quality (valores {df['quality'].min()} a {df['quality'].max()})
   - Registros duplicados: {info_dataset['Duplicados']:,}

2. CALIDAD DE LOS DATOS
--------------------------------------------------------------------------------
   - Valores faltantes: {df.isnull().sum().sum()} ({porcentaje_faltantes.mean():.4f}%)
   - Total de outliers (IQR): {total_outliers_iqr:,}
   - Variables con más outliers: {', '.join(df_outliers.nlargest(3, 'Outliers (IQR)')['Variable'].tolist())}

3. DISTRIBUCIÓN DE LA VARIABLE OBJETIVO
--------------------------------------------------------------------------------
{df_quality.to_string()}

   Categorías:
   - Baja (3-4): {(df['quality_category'] == 'Baja').sum()} ({(df['quality_category'] == 'Baja').sum()/len(df)*100:.2f}%)
   - Media (5-6): {(df['quality_category'] == 'Media').sum()} ({(df['quality_category'] == 'Media').sum()/len(df)*100:.2f}%)
   - Alta (7-9): {(df['quality_category'] == 'Alta').sum()} ({(df['quality_category'] == 'Alta').sum()/len(df)*100:.2f}%)

4. ESTADÍSTICAS DESCRIPTIVAS
--------------------------------------------------------------------------------
{estadisticas[['mean', 'std', 'min', 'max', 'mediana', 'asimetria', 'curtosis']].to_string()}

5. TESTS DE NORMALIDAD
--------------------------------------------------------------------------------
{df_normalidad.to_string()}

6. ANÁLISIS DE CORRELACIONES
--------------------------------------------------------------------------------
   TOP 10 CORRELACIONES MÁS FUERTES:
{df_top_corr.to_string()}

   CORRELACIONES CON CALIDAD:
{df_corr_quality.to_string()}

7. TESTS ESTADÍSTICOS (DIFERENCIAS ENTRE GRUPOS DE CALIDAD)
--------------------------------------------------------------------------------
{df_tests.to_string()}

   Variables con diferencias significativas (Kruskal-Wallis, p<0.05):
   {', '.join(vars_significativas)}

8. ANÁLISIS DE COMPONENTES PRINCIPALES (PCA)
--------------------------------------------------------------------------------
   - Componentes para 80% de varianza: {n_comp_80}
   - Componentes para 95% de varianza: {n_comp_95}
   - Varianza del primer componente: {varianza_explicada[0]*100:.2f}%
   - Varianza de los primeros 3 componentes: {varianza_acumulada[2]*100:.2f}%
   
   Loadings PC1:
{loadings['PC1'].sort_values(key=abs, ascending=False).to_string()}

9. CLUSTERING
--------------------------------------------------------------------------------
   - Número de clusters óptimo (Silhouette): {mejor_k}
   - Clusters utilizados: {n_clusters}
   - Distribución: {dict(df['cluster'].value_counts().sort_index())}
   
   Perfil de clusters (medias):
{perfil_clusters.round(3).to_string()}

10. RESULTADOS DE MACHINE LEARNING - CLASIFICACIÓN
--------------------------------------------------------------------------------
{df_resultados_ml.to_string()}

    Mejor modelo: {df_resultados_ml.iloc[0]['Modelo']}
    - Accuracy: {df_resultados_ml.iloc[0]['Accuracy']:.4f}
    - F1-Score: {df_resultados_ml.iloc[0]['F1-Score']:.4f}
    - Cross-Validation: {df_resultados_ml.iloc[0]['CV Mean']:.4f} (+/- {df_resultados_ml.iloc[0]['CV Std']:.4f})

11. FEATURE IMPORTANCE (RANDOM FOREST)
--------------------------------------------------------------------------------
{df_importancia.to_string()}

12. RESULTADOS DE REGRESIÓN
--------------------------------------------------------------------------------
{df_resultados_reg.to_string()}

   Coeficientes de Regresión Lineal (intercepto: {reg_lr.intercept_:.4f}):
{coef_lr.to_string()}

13. COMPARACIÓN CALIDAD BAJA vs ALTA
--------------------------------------------------------------------------------
{comparacion.round(4).to_string()}

14. ANÁLISIS DE OUTLIERS DETALLADO
--------------------------------------------------------------------------------
{df_outliers.to_string()}

15. OBSERVACIONES CLAVE
--------------------------------------------------------------------------------
   - El alcohol es la variable más importante para predecir calidad ({df_importancia.iloc[0]['Importancia']*100:.1f}% de importancia)
   - Existe una fuerte correlación negativa entre densidad y alcohol ({corr_pearson.loc['density', 'alcohol']:.3f})
   - La mayoría de vinos son de calidad media (5-6), representando {(df['quality_category'] == 'Media').sum()/len(df)*100:.1f}% del dataset
   - {len(vars_significativas)} de {len(columnas_continuas)} variables muestran diferencias significativas entre grupos de calidad
   - El modelo Random Forest logra un accuracy de {df_resultados_ml.iloc[0]['Accuracy']*100:.1f}%
   - Ninguna variable sigue una distribución normal (todas tienen p < 0.05 en tests de normalidad)
   - Los vinos de alta calidad tienen en promedio {comparacion[comparacion['Variable']=='alcohol']['Diferencia (%)'].values[0]:.1f}% más alcohol que los de baja calidad
"""

# Guardar resumen
with open('reports/resumen_analisis_v2.txt', 'w', encoding='utf-8') as f:
    f.write(resumen_completo)

print("   Resumen guardado en: reports/resumen_analisis_v2.txt")

[15/16] Generando resumen...
   Resumen guardado en: reports/resumen_analisis_v2.txt


## 17. GENERAR INSIGHTS CON CLAUDE

In [26]:
print("[16/16] Generando insights con Claude...")

api_key = os.getenv("ANTHROPIC_API_KEY")

if api_key:
    try:
        from anthropic import Anthropic
        
        client = Anthropic(api_key=api_key)
        
        prompt = f"""
Eres un experto en análisis de datos, ciencia de datos y machine learning. 
Analiza el siguiente resumen completo de un análisis de datos sobre calidad de vinos blancos.

Genera un REPORTE EJECUTIVO PROFESIONAL Y DETALLADO que incluya:

1. RESUMEN EJECUTIVO (3-4 párrafos)
   - Descripción del dataset y contexto del problema
   - Calidad general de los datos
   - Principales descubrimientos

2. ANÁLISIS DE LA CALIDAD DE DATOS
   - Evaluación de valores faltantes
   - Análisis de outliers y su impacto
   - Distribución de la variable objetivo y sus implicaciones

3. HALLAZGOS ESTADÍSTICOS CLAVE (5 hallazgos)
   - Patrones y tendencias descubiertos
   - Relaciones significativas entre variables
   - Interpretación de los tests estadísticos

4. ANÁLISIS DE MACHINE LEARNING
   - Evaluación del rendimiento de los modelos
   - Interpretación del feature importance
   - Capacidad predictiva y limitaciones

5. ANÁLISIS DE CLUSTERING Y PCA
   - Interpretación de los clusters encontrados
   - Significado de los componentes principales
   - Patrones de agrupamiento de vinos

6. RECOMENDACIONES DE PREPROCESAMIENTO (5 recomendaciones)
   - Técnicas para mejorar la calidad de datos
   - Estrategias para manejar outliers
   - Transformaciones sugeridas
   - Estrategias para el desbalance de clases

7. LIMITACIONES DEL ANÁLISIS
   - Restricciones del dataset
   - Consideraciones metodológicas
   - Posibles sesgos

8. CONCLUSIONES Y PRÓXIMOS PASOS
   - Síntesis de hallazgos principales
   - Recomendaciones para investigación futura
   - Aplicaciones prácticas

DATOS DEL ANÁLISIS:
{resumen_completo}

Responde en español, de manera profesional y detallada. Usa datos específicos del análisis para respaldar cada punto.
"""

        response = client.messages.create(
            model="claude-sonnet-4-20250514",
            max_tokens=8000,
            messages=[{"role": "user", "content": prompt}]
        )
        
        insights = response.content[0].text
        
        with open('reports/insights_claude_v2.txt', 'w', encoding='utf-8') as f:
            f.write("REPORTE EJECUTIVO - ANÁLISIS DE CALIDAD DE VINOS BLANCOS\n")
            f.write("="*70 + "\n\n")
            f.write(insights)
        
        print("   Insights guardados en: reports/insights_claude_v2.txt")
        print("\n" + "="*70)
        print("REPORTE EJECUTIVO GENERADO")
        print("="*70)
        print(insights[:2000] + "...\n[Ver archivo completo en reports/insights_claude_v2.txt]")
        
    except Exception as e:
        print(f"   Error al generar insights: {e}")
else:
    print("   API key no encontrada. Configura ANTHROPIC_API_KEY en .env")

[16/16] Generando insights con Claude...
   Insights guardados en: reports/insights_claude_v2.txt

REPORTE EJECUTIVO GENERADO
# REPORTE EJECUTIVO: ANÁLISIS DE CALIDAD DE VINOS BLANCOS

## 1. RESUMEN EJECUTIVO

El presente análisis examina un dataset de 4,898 vinos blancos proveniente del UCI Machine Learning Repository, con 11 variables fisicoquímicas y una variable objetivo de calidad (escala 3-9). Los datos presentan una calidad excepcional con ausencia total de valores faltantes, aunque se identificaron 937 registros duplicados y 1,063 outliers distribuidos principalmente en las variables ácido cítrico, cloruros y acidez volátil.

La distribución de la variable objetivo revela un fuerte desbalance de clases, con el 74.62% de los vinos concentrados en las categorías de calidad media (5-6), mientras que solo el 3.74% corresponde a vinos de baja calidad (3-4) y el 21.64% a alta calidad (7-9). Esta distribución asimétrica presenta desafíos significativos para el desarrollo de modelos pr

## FINALIZACIÓN

In [27]:
print("\n" + "="*70)
print("ANÁLISIS COMPLETADO")
print("="*70)
print(
    f"""
ARCHIVOS GENERADOS:
  
  Visualizaciones (outputs/):
    - 01_heatmap_valores_faltantes_v2.png
    - 02_qq_plots_v2.png
    - 03_histogramas_kde_v2.png
    - 04_boxplots_v2.png
    - 05_violin_plots_v2.png
    - 06_barras_quality_v2.png
    - 07_heatmaps_correlacion_v2.png
    - 08_correlacion_con_quality_v2.png
    - 09_pca_varianza_v2.png
    - 10_pca_scatter_v2.png
    - 11_clustering_elbow_v2.png
    - 12_clustering_pca_v2.png
    - 13_confusion_matrix_v2.png
    - 14_comparacion_modelos_v2.png
    - 15_feature_importance_v2.png
    - 16_regression_predictions_v2.png
    
  Reportes (reports/):
    - resumen_analisis_v2.txt
    - insights_claude_v2.txt

MÉTRICAS CLAVE:
  - Dataset: {df.shape[0]:,} registros x {df.shape[1]} columnas
  - Outliers totales: {total_outliers_iqr:,}
  - Mejor modelo: {df_resultados_ml.iloc[0]['Modelo']} (Accuracy: {df_resultados_ml.iloc[0]['Accuracy']*100:.1f}%)
  - Variable más importante: {df_importancia.iloc[0]['Variable']} ({df_importancia.iloc[0]['Importancia']*100:.1f}%)
  - Correlación más fuerte: {top_corr[0][0]} vs {top_corr[0][1]} ({top_corr[0][2]:.3f})
"""
)


ANÁLISIS COMPLETADO

ARCHIVOS GENERADOS:

  Visualizaciones (outputs/):
    - 01_heatmap_valores_faltantes_v2.png
    - 02_qq_plots_v2.png
    - 03_histogramas_kde_v2.png
    - 04_boxplots_v2.png
    - 05_violin_plots_v2.png
    - 06_barras_quality_v2.png
    - 07_heatmaps_correlacion_v2.png
    - 08_correlacion_con_quality_v2.png
    - 09_pca_varianza_v2.png
    - 10_pca_scatter_v2.png
    - 11_clustering_elbow_v2.png
    - 12_clustering_pca_v2.png
    - 13_confusion_matrix_v2.png
    - 14_comparacion_modelos_v2.png
    - 15_feature_importance_v2.png
    - 16_regression_predictions_v2.png

  Reportes (reports/):
    - resumen_analisis_v2.txt
    - insights_claude_v2.txt

MÉTRICAS CLAVE:
  - Dataset: 4,898 registros x 14 columnas
  - Outliers totales: 1,063
  - Mejor modelo: Random Forest (Accuracy: 67.6%)
  - Variable más importante: alcohol (11.7%)
  - Correlación más fuerte: residual sugar vs density (0.839)

